\title{Shift Registers in myHDL (Draft State)}
\author{Steven K Armour}
\maketitle

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Refrances" data-toc-modified-id="Refrances-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Refrances</a></span></li><li><span><a href="#Libraries-and-Helper-functions" data-toc-modified-id="Libraries-and-Helper-functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Libraries and Helper functions</a></span></li><li><span><a href="#myHDL-shift-and-concat-behavior" data-toc-modified-id="myHDL-shift-and-concat-behavior-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>myHDL shift and <code>concat</code> behavior</a></span><ul class="toc-item"><li><span><a href="#shift-(&lt;&lt;/&gt;&gt;)-behavior" data-toc-modified-id="shift-(<</>>)-behavior-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>shift (<code>&lt;&lt;</code>/<code>&gt;&gt;</code>) behavior</a></span><ul class="toc-item"><li><span><a href="#Left-Shift-(&lt;&lt;)" data-toc-modified-id="Left-Shift-(<<)-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Left Shift (<code>&lt;&lt;</code>)</a></span></li><li><span><a href="#Right-Shift-(&gt;&gt;)" data-toc-modified-id="Right-Shift-(>>)-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Right Shift (<code>&gt;&gt;</code>)</a></span></li></ul></li><li><span><a href="#concat-behavior" data-toc-modified-id="concat-behavior-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span><code>concat</code> behavior</a></span></li></ul></li><li><span><a href="#Add-more-basic-shift-registers-here" data-toc-modified-id="Add-more-basic-shift-registers-here-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Add more basic shift registers here</a></span></li><li><span><a href="#Johnson-Counter" data-toc-modified-id="Johnson-Counter-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Johnson Counter</a></span><ul class="toc-item"><li><span><a href="#myHDL-Testing" data-toc-modified-id="myHDL-Testing-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>myHDL Testing</a></span></li><li><span><a href="#Verilog--Code" data-toc-modified-id="Verilog--Code-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Verilog  Code</a></span></li><li><span><a href="#Verilog-Testbench" data-toc-modified-id="Verilog-Testbench-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Verilog Testbench</a></span></li></ul></li><li><span><a href="#Ring-Counter" data-toc-modified-id="Ring-Counter-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Ring Counter</a></span></li><li><span><a href="#To-DO" data-toc-modified-id="To-DO-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>To DO</a></span><ul class="toc-item"><li><span><a href="#prototypeing-ring-counter-sifting-mechnism-with-use-of-concat" data-toc-modified-id="prototypeing-ring-counter-sifting-mechnism-with-use-of-concat-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>prototypeing ring counter sifting mechnism with use of <code>concat</code></a></span></li></ul></li></ul></div>

# Refrances
@misc{myhdl_2017,
title={Johnson Counter},
url={http://www.myhdl.org/docs/examples/jc2.html},
journal={Myhdl.org},
author={myHDL},
year={2017}
}

# Libraries and Helper functions

In [1]:
from myhdl import *
from myhdlpeek import Peeker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sympy import *
init_printing()

import random

#https://github.com/jrjohansson/version_information
%load_ext version_information
%version_information myhdl, myhdlpeek, numpy, pandas, matplotlib, sympy, random

Software,Version
Python,3.6.2 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython,6.2.1
OS,Linux 4.15.0 30 generic x86_64 with debian stretch sid
myhdl,0.10
myhdlpeek,0.0.6
numpy,1.13.3
pandas,0.23.3
matplotlib,2.1.0
sympy,1.1.2.dev
random,The 'random' distribution was not found and is required by the application


In [2]:
#helper  functions to read in the .v and .vhd generated files into python
def VerilogTextReader(loc, printresult=True):
    with open(f'{loc}.v', 'r') as vText:
        VerilogText=vText.read()
    if printresult:
        print(f'***Verilog modual from {loc}.v***\n\n', VerilogText)
    return VerilogText

def VHDLTextReader(loc, printresult=True):
    with open(f'{loc}.vhd', 'r') as vText:
        VerilogText=vText.read()
    if printresult:
        print(f'***VHDL modual from {loc}.vhd***\n\n', VerilogText)
    return VerilogText

In [3]:
CountVal=17
BitSize=int(np.log2(CountVal))+1; BitSize

# myHDL shift and `concat` behavior

## shift (`<<`/`>>`) behavior

### Left Shift (`<<`)

In [4]:
#Left Shift test with intbv
#intialize 
q=intbv(5)[4:]
bin(q, 4)

'0101'

In [5]:
#demenstrate left shifting with intbv
for i in range(9):
    q=q<<2
    print(f'{i}:q={q}, {bin(q, 4)}', type(q))

0:q=14, 10100 <class 'myhdl._intbv.intbv'>
1:q=50, 1010000 <class 'myhdl._intbv.intbv'>
2:q=140, 101000000 <class 'myhdl._intbv.intbv'>
3:q=500, 10100000000 <class 'myhdl._intbv.intbv'>
4:q=1400, 1010000000000 <class 'myhdl._intbv.intbv'>
5:q=5000, 101000000000000 <class 'myhdl._intbv.intbv'>
6:q=14000, 10100000000000000 <class 'myhdl._intbv.intbv'>
7:q=50000, 1010000000000000000 <class 'myhdl._intbv.intbv'>
8:q=140000, 101000000000000000000 <class 'myhdl._intbv.intbv'>


In [6]:
#Left Shift test with modbv
#intialize 
q=modbv(5)[4:]
bin(q, 4)

'0101'

In [7]:
#demenstrate left shifting with modbv
for i in range(9):
    q=q<<2
    print(f'{i}:q={q}, {bin(q, 4)}', type(q))

0:q=14, 10100 <class 'myhdl._intbv.intbv'>
1:q=50, 1010000 <class 'myhdl._intbv.intbv'>
2:q=140, 101000000 <class 'myhdl._intbv.intbv'>
3:q=500, 10100000000 <class 'myhdl._intbv.intbv'>
4:q=1400, 1010000000000 <class 'myhdl._intbv.intbv'>
5:q=5000, 101000000000000 <class 'myhdl._intbv.intbv'>
6:q=14000, 10100000000000000 <class 'myhdl._intbv.intbv'>
7:q=50000, 1010000000000000000 <class 'myhdl._intbv.intbv'>
8:q=140000, 101000000000000000000 <class 'myhdl._intbv.intbv'>


As can be seen, Left shifting tacks on a number of zeros equivalent to the shift increment to the end of the binary expression for the value. This then increases the size of the needed register that the resulting value needs to set into for each left shift that does not undergo right bit cutoff

### Right Shift (`>>`)

In [8]:
#Right Shift test with intbv
#intialize 
q=intbv(5)[4:].signed()
bin(q, 4)

'0101'

In [9]:
#demenstrate right shifting with intbv
for i in range(9):
    q=q>>2
    print(f'{i}:q={q}, {bin(q, 4)}', type(q))

0:q=1, 0001 <class 'myhdl._intbv.intbv'>
1:q=0, 0000 <class 'myhdl._intbv.intbv'>
2:q=0, 0000 <class 'myhdl._intbv.intbv'>
3:q=0, 0000 <class 'myhdl._intbv.intbv'>
4:q=0, 0000 <class 'myhdl._intbv.intbv'>
5:q=0, 0000 <class 'myhdl._intbv.intbv'>
6:q=0, 0000 <class 'myhdl._intbv.intbv'>
7:q=0, 0000 <class 'myhdl._intbv.intbv'>
8:q=0, 0000 <class 'myhdl._intbv.intbv'>


In [10]:
#Right Shift test with modbv
#intialize 
q=modbv(5)[4:]
bin(q, 4)

'0101'

In [11]:
#demenstrate right shifting with modbv
for i in range(9):
    q=q>>2
    print(f'{i}:q={q}, {bin(q, 4)}', type(q))

0:q=1, 0001 <class 'myhdl._intbv.intbv'>
1:q=0, 0000 <class 'myhdl._intbv.intbv'>
2:q=0, 0000 <class 'myhdl._intbv.intbv'>
3:q=0, 0000 <class 'myhdl._intbv.intbv'>
4:q=0, 0000 <class 'myhdl._intbv.intbv'>
5:q=0, 0000 <class 'myhdl._intbv.intbv'>
6:q=0, 0000 <class 'myhdl._intbv.intbv'>
7:q=0, 0000 <class 'myhdl._intbv.intbv'>
8:q=0, 0000 <class 'myhdl._intbv.intbv'>


As can be seen, the right shift moves values (shifts) to the right all the values to the right by the shift increment while preserving the length of the register that is being shifted. While this means that overflow is not going to be in encountered with the use of the right shift operation. It trades that vulnerability for information loss as any information carried in the leftmost bits gets lost as it is shifted right beyond of the length of the register

## `concat` behavior

to do

# Add more basic shift registers here 

# Johnson Counter
The Johnson Counter is implemented in this notebook about shift registers since in reality, a Johnson Counter is a shift register. The following are aspects of Johnson Counter by Sougata Bhattacharjee [https://www.quora.com/What-is-the-difference-between-a-Johnson-counter-and-a-ring-counter]

\begin{itemize}
\item In a Johnson Counter the output bar or Q(bar) of the last flip-flop is connected to the input of the first flip-flop

\item If $n$ is the number of the flip-flop used, then the total number of states used is $2n$.

\item The Johnson Counter is also known as walking counter, switching tail counter and is mostly used in phase shift or function generator.

\item The decoding circuit is complex as compared to a ring counter.

\item If input frequency is $f$ in Johnson Counter,then the output is $\dfrac{f}{2n}$.

\item The total number of unused states in Johnson Counter is $(2^n - 2n)$

\item The main problem with Johnson counter is that once it enters into an unused state it is in a lockout state

\end{itemize}


For a four-bit Johnson Counter, the next state diagram is given by the following table from [wikipedga](https://en.wikipedia.org/wiki/Ring_counter)

\begin{figure}
\centerline{\includegraphics[width=1cm]{JohnsonCounterNSTable.png}}
\caption{\label{fig:JohnsonTable} 4-bit Ring Counter next state table from [wikipedia](https://en.wikipedia.org/wiki/Ring_counter#Four-bit_ring-counter_sequences)}
\end{figure}

whereupon examining the next state table for the Johnson counter we can see why a Johnson counter is also called a Mobius counter



In [12]:
#Create the Direction States for Johnson Counter
DirStates=enum('Left','Halt','Right')
print(f"`Left` state repersentation is {bin(DirStates.Left)}")
print(f"`Halt` state repersentation is {bin(DirStates.Halt)}")
print(f"`Right` state repersentation is {bin(DirStates.Right)}")

`Left` state repersentation is 0
`Halt` state repersentation is 1
`Right` state repersentation is 10


In [13]:
@block
def JohnsonCount3(Dir, q, clk, rst):
    """
    Based of the `jc2` exsample from the myHDL website 
    http://www.myhdl.org/docs/examples/jc2.html
    
    Input:
        Dir(state): Left,Right, Halt Direction States
        clk(bool): input clock
        rst(bool): reset signal
    
    Ouput:
        q(bitVec): the values in the D flip flops(aka counter)
    """
    
    q_i=Signal(intbv(0)[len(q):])
    @always(clk.posedge, rst.negedge)
    def JCStateMachine():
        #moore state machine
        if rst:
            q_i.next=0
        
        elif Dir==DirStates.Left:
            #set bit slice from left most to one from the right
            #from bit slice from one to the left to the right most
            q_i.next[len(q_i):1]=q_i[len(q_i)-1:]
            #set next right most bit to negated one to the left bit
            q_i.next[0]=not q_i[len(q_i)-1]
        
        elif Dir==DirStates.Halt:
            #create circular stop
            q_i.next=q_i
        
        elif Dir==DirStates.Right:
            #set bit slice from one from the left to right most
            #from bit slice left most bit to one from the right
            q_i.next[len(q_i)-1:]=q_i[len(q_i):1]
            #set next one bit from the right to be negated left most bit  
            q_i.next[len(q_i)-1]=not q_i[0]
            
            
    
    @always_comb
    def OuputBuffer():
        q.next=q_i
    
    return instances()
    

## myHDL Testing

In [14]:
BitSize=4
Peeker.clear()
clk=Signal(bool(0)); Peeker(clk, 'clk')
rst=Signal(bool(0)); Peeker(rst, 'rst')
q=Signal(intbv(0)[BitSize:]); Peeker(q, 'q')
Dir=Signal(DirStates.Right); Peeker(Dir, 'Dir')

DUT=JohnsonCount3(Dir, q, clk, rst)

def JohnsonCount3_TB():
    """
    myHDL only Testbench for `UpDown_Counter` module
    """
    @always(delay(1))
    def ClkGen():
        clk.next=not clk
    
    @instance
    def stimules():
        i=0
        while True:
            if i==2*2*BitSize:
                Dir.next=DirStates.Left
            elif i==4*2*BitSize:
                rst.next=1
            elif i==4*2*BitSize+1:
                rst.next=0
            elif i==4*2*BitSize+2:
                Dir.next=DirStates.Halt
            
            
            if i==5*2*BitSize:
                raise StopSimulation()
                
            i+=1
            yield clk.posedge
    
    return instances()

sim=Simulation(DUT, JohnsonCount3_TB(), *Peeker.instances()).run()

In [15]:
Peeker.to_wavedrom()

In [16]:
JohnsonCount3Data=Peeker.to_dataframe()
JohnsonCount3Data=JohnsonCount3Data[JohnsonCount3Data['clk']==1]
JohnsonCount3Data.drop('clk', axis=1, inplace=True)
JohnsonCount3Data.reset_index(drop=True, inplace=True)
JohnsonCount3Data

,Dir,q,rst
0,Right,8,0
1,Right,12,0
2,Right,14,0
3,Right,15,0
4,Right,7,0
5,Right,3,0
6,Right,1,0
7,Right,0,0
8,Right,8,0
9,Right,12,0


In [17]:
JohnsonCount3Data['q']=JohnsonCount3Data['q'].apply(lambda x:bin(x, BitSize))
JohnsonCount3Data

,Dir,q,rst
0,Right,1000,0
1,Right,1100,0
2,Right,1110,0
3,Right,1111,0
4,Right,0111,0
5,Right,0011,0
6,Right,0001,0
7,Right,0000,0
8,Right,1000,0
9,Right,1100,0


In [18]:
JohnsonCount3Data[JohnsonCount3Data['Dir']==DirStates.Right]

,Dir,q,rst
0,Right,1000,0
1,Right,1100,0
2,Right,1110,0
3,Right,1111,0
4,Right,0111,0
5,Right,0011,0
6,Right,0001,0
7,Right,0000,0
8,Right,1000,0
9,Right,1100,0


In [19]:
JohnsonCount3Data[JohnsonCount3Data['Dir']==DirStates.Left]

,Dir,q,rst
15,Left,0000,0
16,Left,0001,0
17,Left,0011,0
18,Left,0111,0
19,Left,1111,0
20,Left,1110,0
21,Left,1100,0
22,Left,1000,0
23,Left,0000,0
24,Left,0001,0


In [20]:
JohnsonCount3Data[JohnsonCount3Data['Dir']==DirStates.Halt]

,Dir,q,rst
33,Halt,0011,0
34,Halt,0011,0
35,Halt,0011,0
36,Halt,0011,0
37,Halt,0011,0
38,Halt,0011,0


## Verilog  Code

In [21]:
DUT.convert()
VerilogTextReader('JohnsonCount3');

***Verilog modual from JohnsonCount3.v***

 // File: JohnsonCount3.v
// Generated by MyHDL 0.10
// Date: Tue Aug 14 07:16:07 2018


`timescale 1ns/10ps

module JohnsonCount3 (
    Dir,
    q,
    clk,
    rst
);
// Based of the `jc2` exsample from the myHDL website 
// http://www.myhdl.org/docs/examples/jc2.html
// 
// Input:
//     Dir(state): Left,Right, Halt Direction States
//     clk(bool): input clock
//     rst(bool): reset signal
// 
// Ouput:
//     q(bitVec): the values in the D flip flops(aka counter)

input [1:0] Dir;
output [3:0] q;
wire [3:0] q;
input clk;
input rst;

reg [3:0] q_i;



always @(posedge clk, negedge rst) begin: JOHNSONCOUNT3_JCSTATEMACHINE
    if (rst) begin
        q_i <= 0;
    end
    else if ((Dir == 2'b00)) begin
        q_i[4-1:1] <= q_i[(4 - 1)-1:0];
        q_i[0] <= (!q_i[(4 - 1)]);
    end
    else if ((Dir == 2'b01)) begin
        q_i <= q_i;
    end
    else if ((Dir == 2'b10)) begin
        q_i[(4 - 1)-1:0] <= q_i[4-1:1];
        q_i[(4 - 1)] 

\begin{figure}
\centerline{\includegraphics[width=10cm]{JohnsonCount3RTL.png}}
\caption{\label{fig:clkDivRTL} JohnsonCount3 RTL schematic; Xilinx Vivado 2017.4}
\end{figure}

\begin{figure}
\centerline{\includegraphics[width=10cm]{JohnsonCount3SYN.png}}
\caption{\label{fig:clkDivRTL} JohnsonCount3 Synthesized Schematic; Xilinx Vivado 2017.4}
\end{figure}

## Verilog Testbench

In [22]:
@block
def JohnsonCount3_TBV():
    """
    myHDL -> Verilog Testbench for `UpDown_Counter` module
    """
    
    clk=Signal(bool(0))
    rst=Signal(bool(0))
    q=Signal(intbv(0)[BitSize:])
    Dir=Signal(DirStates.Right)
    
    @always_comb
    def print_data():
        print(clk, rst, q, Dir)

    DUT=JohnsonCount3(Dir, q, clk, rst)

    @instance
    def clk_signal():
        while True:
            clk.next = not clk
            yield delay(1)
    
    @instance
    def stimules():
        i=0
        while True:
            if i==2*2*BitSize:
                Dir.next=DirStates.Left
            elif i==4*2*BitSize:
                rst.next=1
            elif i==4*2*BitSize+1:
                rst.next=0
            elif i==4*2*BitSize+2:
                Dir.next=DirStates.Halt
            else:
                pass
            
            
            if i==5*2*BitSize:
                raise StopSimulation()
                
            i+=1
            yield clk.posedge
    
    return instances()

TB=JohnsonCount3_TBV()
TB.convert(hdl="Verilog", initial_values=True)
VerilogTextReader('JohnsonCount3_TBV');

<class 'myhdl._Signal._Signal'> <class '_ast.Name'>
<class 'myhdl._Signal._Signal'> <class '_ast.Name'>
<class 'myhdl._Signal._Signal'> <class '_ast.Name'>
<class 'myhdl._Signal._Signal'> <class '_ast.Name'>
***Verilog modual from JohnsonCount3_TBV.v***

 // File: JohnsonCount3_TBV.v
// Generated by MyHDL 0.10
// Date: Tue Aug 14 07:16:07 2018


`timescale 1ns/10ps

module JohnsonCount3_TBV (

);
// myHDL -> Verilog Testbench for `UpDown_Counter` module


wire [3:0] q;
reg clk = 0;
reg rst = 0;
reg [1:0] Dir = 2'b10;
reg [3:0] JohnsonCount30_0_q_i = 0;



always @(clk, rst, q, Dir) begin: JOHNSONCOUNT3_TBV_PRINT_DATA
    $write("%h", clk);
    $write(" ");
    $write("%h", rst);
    $write(" ");
    $write("%h", q);
    $write(" ");
    $write("%h", Dir);
    $write("\n");
end


always @(posedge clk, negedge rst) begin: JOHNSONCOUNT3_TBV_JOHNSONCOUNT30_0_JCSTATEMACHINE
    if (rst) begin
        JohnsonCount30_0_q_i <= 0;
    end
    else if ((Dir == 2'b00)) begin
        JohnsonCount3

# Ring Counter
Like the Johnson Counter, which is the Mobius ring version of the Ring Counter, a Ring Counter is, in reality, a shift register The following are 
aspects of Ring Counter by Sougata Bhattacharjee [https://www.quora.com/What-is-the-difference-between-a-Johnson-counter-and-a-ring-counter]

\begin{itemize}
\item In a ring counter, the output of the last flip-flop is connected to the input of the first flip-flop.

\item If $n$ is the number of flip-flops that are used in ring counter, the number of possible states are also $n$.That means the number of states is equal to the number of flip-flops used.

\item A Ring counter is mostly used in Successive approximation type ADC and stepper motor control. 

\item Decoding is easy in a ring counter as the number of states is equal to the number of flip-flops.

\item If the input frequency to a ring counter is $f$ then the output frequency $\dfrac{f}{n}$.

\item The total number of unused states in the ring counter is $(2^n - n)$.

\end{itemize}

For a four-bit ring counter, the next state diagram is given by the following table from [wikipedga](https://en.wikipedia.org/wiki/Ring_counter)

\begin{figure}
\centerline{\includegraphics[width=1cm]{RingCounterNSTable.png}}
\caption{\label{fig:RingTable} 4-bit Ring Counter next state table from [wikipedia](https://en.wikipedia.org/wiki/Ring_counter#Four-bit_ring-counter_sequences)}
\end{figure}


# To DO
## prototypeing ring counter sifting mechnism with use of `concat`

In [23]:
#test with intbv
#intialize 
q=intbv(1)[4:]
bin(q, 4)

'0001'

In [24]:
#demenstrate right shifting with intbv
for i in range(9):
    q=concat(q[0],q[4:1])
    print(f'{i}:q={q}, {bin(q, 4)}')

0:q=8, 1000
1:q=4, 0100
2:q=2, 0010
3:q=1, 0001
4:q=8, 1000
5:q=4, 0100
6:q=2, 0010
7:q=1, 0001
8:q=8, 1000


right shifting using `intbv` works

In [25]:
#test with modbv
#intialize 
q=modbv(1)[4:]
bin(q, 4)

'0001'

In [26]:
#demenstrate right shifting with modbv
for i in range(9):
    q=concat(q[0],q[4:1])
    print(f'{i}:q={q}, {bin(q, 4)}')

0:q=8, 1000
1:q=4, 0100
2:q=2, 0010
3:q=1, 0001
4:q=8, 1000
5:q=4, 0100
6:q=2, 0010
7:q=1, 0001
8:q=8, 1000


using `modbv` works as well as `intbv` for right shifting

In [27]:
#test with intbv
#intialize 
q=intbv(1)[4:]
bin(q, 4)

'0001'

In [28]:
#demenstrate left shifting with intbv
for i in range(9):
    q=concat(q[3:0],q[4])
    print(f'{i}:q={q}, {bin(q, 4)}')

0:q=2, 0010
1:q=4, 0100
2:q=8, 1000
3:q=0, 0000
4:q=0, 0000
5:q=0, 0000
6:q=0, 0000
7:q=0, 0000
8:q=0, 0000


there is a proplom in the shift from `1000` back to `0001` 

In [29]:
#test with modbv
#intialize 
q=modbv(1)[4:]
bin(q, 4)

'0001'

In [30]:
#demenstrate left shifting with modbv
for i in range(9):
    q=concat(q[3:0],q[4])
    print(f'{i}:q={q}, {bin(q, 4)}')

0:q=2, 0010
1:q=4, 0100
2:q=8, 1000
3:q=0, 0000
4:q=0, 0000
5:q=0, 0000
6:q=0, 0000
7:q=0, 0000
8:q=0, 0000


the proplom still presists in the shift from `1000` back to `0001`